In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

In [5]:
df = pd.read_csv("spam.csv",encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
#delete column 
df = df.drop('Unnamed: 2', axis=1)
df = df.drop('Unnamed: 3', axis=1)
df = df.drop('Unnamed: 4', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [7]:
#checking null 
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [8]:
#change column name
df.rename(columns={'v1': 'type', 'v2': 'text'}, inplace=True)

print(df)

      type                                               text
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will Ì_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [9]:
# encoding the type to 0,1
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['type'])

df['label'] = le.transform(df['type'])
df.head()

,type,text,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [10]:
# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [11]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
# Tokenize and pad the SMS messages for BERT input
max_length = 128  
train_tokens = tokenizer(list(train_df['text']), padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
test_tokens = tokenizer(list(test_df['text']), padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')

In [13]:
# Convert labels to numpy arrays
train_labels = np.array(train_df['label'])
test_labels = np.array(test_df['label'])

In [14]:
# Load the pre-trained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [15]:
# Build the LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=len(tokenizer.vocab), output_dim=128, input_length=max_length))
lstm_model.add(Bidirectional(LSTM(64)))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1, activation='sigmoid'))

In [16]:
# Combine BERT and LSTM models
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)
bert_output = bert_model(input_ids)[1]  # Using [1] to get the BERT pooled output
lstm_output = lstm_model(input_ids)
combined_output = tf.keras.layers.concatenate([bert_output, lstm_output])
output = Dense(1, activation='sigmoid')(combined_output)

In [17]:
# Create a combined model
combined_model = tf.keras.models.Model(inputs=input_ids, outputs=output)

In [19]:
# Compile the combined model
combined_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Train the model
combined_model.fit(train_tokens['input_ids'], train_labels, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
112/112 [==============================] - 1629s 15s/step - loss: 0.2397 - accuracy: 0.9066 - val_loss: 0.1073 - val_accuracy: 0.9686
Epoch 2/5
112/112 [==============================] - 1721s 15s/step - loss: 0.0625 - accuracy: 0.9795 - val_loss: 0.1262 - val_accuracy: 0.9507
Epoch 3/5
112/112 [==============================] - 1711s 15s/step - loss: 0.0359 - accuracy: 0.9902 - val_loss: 0.0647 - val_accuracy: 0.9832
Epoch 4/5
112/112 [==============================] - 1686s 15s/step - loss: 0.0239 - accuracy: 0.9935 - val_loss: 0.0798 - val_accuracy: 0.9809
Epoch 5/5
112/112 [==============================] - 1686s 15s/step - loss: 0.0149 - accuracy: 0.9964 - val_loss: 0.0697 - val_accuracy: 0.9843


In [21]:
# Evaluate the model on the test set
loss, accuracy = combined_model.evaluate(test_tokens['input_ids'], test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

35/35 [==============================] - 104s 3s/step - loss: 0.0339 - accuracy: 0.9937
Test Loss: 0.033937301486730576, Test Accuracy: 0.9937219619750977
